In [115]:
import psycopg2
import requests
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
import json
from datetime import datetime
import json
import random

random.seed( 30 )

In [116]:
authorized_type_tournement = ['Grand Chelem','ATP']

with open('data/tournaments.json') as json_file:
    data = json.load(json_file)

authorized_tournement = list(data.keys())

In [117]:
all_ids = {}

matches = requests.get('http://api.unicdn.net/v1/feeds/sportsbook/group/1000093193.json?app_id=506c35d4&app_key=9d47ace92665dd700a19194f822f858a&local=fr_CH&rangeStart=0&outComeSortBy=lexical&outComeSortDir=desc&type=2&rangeSize=100&includeparticipants=true&site=fr.unibet.com&BetOfferType=name').json()
all_tournementsKindred = [tournement['name'] for tournement  in matches['group']['groups']]

for typeOfTournement in matches['group']['groups']:
    if typeOfTournement['name'] in authorized_type_tournement:
        for tournement in typeOfTournement['groups']:
            if 'Femmes' not in tournement['name'] and 'Doubles' not in tournement['name'] :
                all_ids[tournement['name']] = tournement['id']
all_ids

{'Open de France': 1000096468,
 'Wimbledon': 1000096467,
 'US Open': 1000096466,
 'Spécial': 1000096479,
 'Buenos Aires': 1000096214,
 'New York': 2000113039,
 'Rotterdam': 1000096215}

In [118]:
id_events = {}
for tournement, _id in all_ids.items():
    detailsTournement = requests.get('http://api.unicdn.net/v1/feeds/sportsbookv2/event/group/'+str(_id)+'.json?app_id=506c35d4&app_key=9d47ace92665dd700a19194f822f858a&includeparticipants=false').json()
    for events in detailsTournement['events']:
        id_events[events['id']] = tournement
        
id_events

{1005775288: 'Open de France',
 1005775292: 'Wimbledon',
 1005779650: 'US Open',
 1006120223: 'Spécial',
 1006132949: 'Buenos Aires',
 1006134460: 'Buenos Aires',
 1006134462: 'Buenos Aires',
 1006134463: 'Buenos Aires',
 1006132947: 'Buenos Aires',
 1006132946: 'Buenos Aires',
 1006136407: 'Buenos Aires',
 1006136833: 'Buenos Aires',
 1006136470: 'New York',
 1006132955: 'New York',
 1006136471: 'New York',
 1006132958: 'New York',
 1006132953: 'New York',
 1006132952: 'New York',
 1006136472: 'New York',
 1006136925: 'New York',
 1006136858: 'New York',
 1006131448: 'Rotterdam',
 1006131451: 'Rotterdam',
 1006134174: 'Rotterdam',
 1006131441: 'Rotterdam',
 1006131443: 'Rotterdam',
 1006136246: 'Rotterdam',
 1006131445: 'Rotterdam',
 1006137706: 'Rotterdam',
 1006131738: 'Rotterdam',
 1006137577: 'Rotterdam'}

In [119]:
matchesToPredict = {}

for id_event,tournement in id_events.items():

    currentRequestEvent = requests.get('http://api.unicdn.net/v1/feeds/sportsbookv2/betoffer/event/'+str(id_event)+'.json?app_id=506c35d4&app_key=9d47ace92665dd700a19194f822f858a&includeparticipants=false&outComeSortBy=lexical&outComeSortDir=desc').json()
    betOffers = currentRequestEvent['betOffers']
    events = currentRequestEvent['events']
    matchs = {}

    for event in events :
        matchs[event['id']] = event["englishName"]

    ids = list(matchs.keys())
    
    for bet in betOffers:
        if bet['eventId'] in ids:
            if 'Match Odds' in bet['criterion']['label']:

                joueur0 = min(matchs[bet['eventId']].split('-')[0].rstrip().lstrip().lower().replace(',','').split(' '), matchs[bet['eventId']].split('-')[1].rstrip().lstrip().lower().replace(',','').split(' '))
                joueur1 = max(matchs[bet['eventId']].split('-')[0].rstrip().lstrip().lower().replace(',','').split(' '), matchs[bet['eventId']].split('-')[1].rstrip().lstrip().lower().replace(',','').split(' '))
                try:
                    matchesToPredict[joueur0[0]+joueur1[0]] = {
                                             'tournement': tournement,
                                             'start': events[0]['start'],
                                             'joueur0': joueur0[1] + " " + joueur0[0],
                                             'joueur1': joueur1[1] + " " + joueur1[0],
                                             'oddJoueur0': 1 + eval(bet['outcomes'][0]["oddsFractional"]),
                                             'oddJoueur1': 1 + eval(bet['outcomes'][1]["oddsFractional"])}
                except:
                    pass

matchesToPredict

{'carballescecchinato': {'tournement': 'Buenos Aires',
  'start': '2020-02-11T17:33:55Z',
  'joueur0': 'baena carballes',
  'joueur1': 'marco cecchinato',
  'oddJoueur0': 2.4,
  'oddJoueur1': 1.52},
 'bagnismartin': {'tournement': 'Buenos Aires',
  'start': '2020-02-11T17:36:13Z',
  'joueur0': 'facundo bagnis',
  'joueur1': 'andrej martin',
  'oddJoueur0': 6.25,
  'oddJoueur1': 1.1111111111111112},
 'kovalikmoutet': {'tournement': 'Buenos Aires',
  'start': '2020-02-11T17:37:40Z',
  'joueur0': 'jozef kovalik',
  'joueur1': 'corentin moutet',
  'oddJoueur0': 3.9,
  'oddJoueur1': 1.24},
 'martinezverdasco': {'tournement': 'Buenos Aires',
  'start': '2020-02-11T19:10:00Z',
  'joueur0': 'portero martinez',
  'joueur1': 'fernando verdasco',
  'oddJoueur0': 1.55,
  'oddJoueur1': 2.4},
 'cerundolodjere': {'tournement': 'Buenos Aires',
  'start': '2020-02-11T20:25:00Z',
  'joueur0': 'francisco cerundolo',
  'joueur1': 'laslo djere',
  'oddJoueur0': 1.35,
  'oddJoueur1': 3.2},
 'coricmonteiro':

In [120]:
all_players = pd.read_csv('data/all_players.csv')
all_tournements = pd.read_csv('data/all_tournaments.csv')
df2020 = pd.read_csv('data/all_tournaments_with_stats_for_DNN2020.csv').dropna()
all_tournaments_with_stats_for_DNN = pd.read_csv('data/all_tournaments_with_stats_for_DNN.csv').dropna()

df2020.label = df2020.label.replace('joueur0',0)
df2020.label = df2020.label.replace('joueur1',1)

df2020.pointsDropping0 = df2020.pointsDropping0.apply(lambda x: x.replace(',','.'))
df2020.pointsDropping1 = df2020.pointsDropping1.apply(lambda x: x.replace(',','.'))

all_tournaments_with_stats_for_DNN.RateFace2Face0 = all_tournaments_with_stats_for_DNN.RateFace2Face0/100
all_tournaments_with_stats_for_DNN.RateFace2Face1 = all_tournaments_with_stats_for_DNN.RateFace2Face1/100

all_tournaments_with_stats_for_DNN = pd.concat([all_tournaments_with_stats_for_DNN, df2020], axis=0)

all_tournements['surface'] = [surface.split(' ')[1].lower() for surface in all_tournements.surfaceInout]
all_tournements['date'] = [(datetime.strptime(date.split('-')[0].lstrip().rstrip(), '%Y.%m.%d').date()) for date in all_tournements.date]
all_tournements = all_tournements[(all_tournements['date'] > datetime.strptime('2017.01.01', '%Y.%m.%d').date())]

all_players['surface'] = [url.split('=')[2] for url in all_players.url]

/Users/matrousseau/.local/share/virtualenvs/Pronoo-RxTA2Kor/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (37,65) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [121]:
def getFormPlayer(player, year):
    
    counter = 0
    
    df = all_tournaments_with_stats_for_DNN[(all_tournaments_with_stats_for_DNN.player0==player) | (all_tournaments_with_stats_for_DNN.player1==player) 
                                            & (all_tournaments_with_stats_for_DNN.year<year)]
    
    forme = 0
    
    if len(df)>3:
        
        try:
            forme = int(df[df.player0 == player][0:4].player0.value_counts())/4       
            return forme
        
        except:
            return np.NaN
        
    else:
        
        try:
            return int(df[df.player0 == player][0:len(df)].player0.value_counts())/len(df)
        except:
            counter +=1
            
            if counter%1000 == 0:
                print(counter)
            return 0.5

In [122]:
all_players_name = list(all_players.player)

all_match_to_predict = pd.DataFrame([])

from fuzzywuzzy import fuzz

def get_corresponding_player(player, all_players_odds):
    best_ratio = 0
    best_word = ""    
    for playerodds in all_players_odds:
        if isinstance(playerodds, str):
            current_ratio = fuzz.ratio(playerodds, player)
            if current_ratio>best_ratio:
                best_ratio = current_ratio
                best_word = playerodds

    return best_word

def getInfoTournois(name):
    with open('data/tournaments.json') as json_file:
        data = json.load(json_file)        
    return data[name]['surface'], data[name]['inOut'], data[name]['points']

In [123]:
def getCorrespondingStats(player, surface):
    
    df = all_players[(all_players.player == get_corresponding_player(player,all_players_name)) & (all_players.surface == surface)]
    df = df.replace('[]',np.NaN)
    df = df.sort_values(by=['year'], ascending=False)
        
    try:
        
        df = df.dropna()
        
        if len(df) > 0:
            df = all_players[(all_players.player == get_corresponding_player(player,all_players_name)) & (all_players.surface == surface)]
            if list(df.isnull().sum(axis=1))[0] > 0:
                df = all_players[(all_players.player == get_corresponding_player(player,all_players_name)) & (all_players.surface == "hard")]
        return df.iloc[[-2]]
    
    except:
        pass


def getPlayersConfrontation(player0, player1, surface, ranking0, ranking1):
    
    player0DictName = get_corresponding_player(player0,all_players_name)
    player1DictName = get_corresponding_player(player1,all_players_name)
    
    df0 = all_tournements[(all_tournements.winnerName == player0DictName) & (all_tournements.looserName == player1DictName) & (all_tournements.surface == surface)]
    df1 = all_tournements[(all_tournements.winnerName == player1DictName) & (all_tournements.looserName == player0DictName) & (all_tournements.surface == surface)]
    try:
        df = pd.concat([df0,df1],axis=0).sort_values(by=['date'], ascending=False).reset_index().iloc[0]
    except:
        
        ranking = getGapRank(ranking0, ranking1)
        player0ranking = ranking[0]
        player1ranking = ranking[1]
        
        if ranking1.values[0]<ranking0.values[0]:
            player0ranking = ranking[1]
            player1ranking = ranking[0]
        
        
        return {
                "player1":{
                    "player1Rate":player1ranking,
                    "player1RateSurface":player1ranking,
                },
                "player0":{
                    "player0Rate":player0ranking,
                    "player0RateSurface":player0ranking
                }
            }
    
    df = df.replace('[]',np.NaN)
        
    if list(df.isnull())[0] > 0:
        
        ranking = getGapRank(ranking0, ranking1)
        player0ranking = ranking[0]
        player1ranking = ranking[1]
        
        if ranking1.values[0]<ranking0.values[0]:
            player0ranking = ranking[1]
            player1ranking = ranking[0]
        
        return {
                "player1":{
                    "player1Rate":player1Rate,
                    "player1RateSurface":player1Rate,
                },
                "player0":{
                    "player0Rate":player0Rate,
                    "player0RateSurface":player0Rate,
                }
            }
    
    else:    
        if df.winnerName == player0DictName:
            return {
                "player0":{
                    "player0Rate":df.winnerRate,
                    "player0RateSurface":df.winnerRateSurface,
                },
                "player1":{
                    "player1Rate":df.looserRate,
                    "player1RateSurface":df.looserRateSurface,
                }
            }
        else:
            return {
                "player1":{
                    "player1Rate":df.winnerRate,
                    "player1RateSurface":df.winnerRateSurface,
                },
                "player0":{
                    "player0Rate":df.looserRate,
                    "player0RateSurface":df.looserRateSurface,
                }
            }

    
    
    return 

def getGapRank(rank0, rank1):
    beta = abs(rank0.values[0]-rank1.values[0])/300
    res = [(1-beta)/2,(1+beta)/2]
    return max(res),min(res)

def returnInOut(inOut):
    
    dummies = pd.get_dummies(['Indoor','Outdoor'])
    
    if 'Indoor' in inOut:
        return dummies.iloc[0]
    else:
        return dummies.iloc[1]

def returnPlays(play, playerIndex):
    
    dummies = pd.get_dummies(['Left-Handed','Right-Handed', 'Left-Handed.1', 'Right-Handed.1'])
    
    if 'left' in play.lower():
        if playerIndex == 0:
            return dummies.iloc[0][['Left-Handed','Right-Handed']]
        else:
            return dummies.iloc[2][['Left-Handed.1','Right-Handed.1']]
    else:
        if playerIndex == 0:
            return dummies.iloc[1][['Left-Handed','Right-Handed']]
        else:
            return dummies.iloc[3][['Left-Handed.1','Right-Handed.1']]

def returnSurface(surface):
    
    dummies = pd.get_dummies(['Carpet', 'Clay', 'Grass', 'Hard'])
    
    if surface.lower()=='carpet':
        return dummies.iloc[0]
    elif surface.lower()=='clay':
            return dummies.iloc[1]
    elif surface.lower()=='grass':
            return dummies.iloc[2]
    else:
        return dummies.iloc[3]
    
def getFormeAndRankMean(player):
    df = all_tournements_forDNN[(all_tournements_forDNN.player0 == player) | (all_tournements_forDNN.player1 == player)].iloc[-1]

    if (df.label == 0 and df.player0 == player):
        return df.Forme0, df.WinMean0

    else:
        return df.Forme1, df.WinMean1

In [124]:
def buildDatasetForPrediction():
    all_match_to_predict = pd.DataFrame()
    all_plays = pd.DataFrame()
    allSurface = pd.DataFrame()
    allInOut = pd.DataFrame()
    allTypeTournement = pd.DataFrame()
    matchSelected = []
    AllForme0 = []
    AllForme1 = []
    AllWinMean0 = []
    AllWinMean1 = []

    allPlayers = list(set(list(set(all_tournements_forDNN.player0)) + list(set(all_tournements_forDNN.player1))))

    for currentMatch in matchesToPredict.values():

        try:
            surface, inout, typeTournement = getInfoTournois(currentMatch['tournement'])

            joueur0 = min(currentMatch['joueur0'], currentMatch['joueur1'])
            joueur1 = max(currentMatch['joueur0'], currentMatch['joueur1'])

            print("joueur 0", joueur0)
            print("joueur 1", joueur1)

            joueur0Stats = getCorrespondingStats(joueur0, surface)

            forme0, winMean0 = getFormeAndRankMean(get_corresponding_player(joueur0, allPlayers))

            joueur0Stats.columns = ['player0', 'aces0', 'doubleFautes0', 'firstServe0', 'firstServePointsWon0',
                                    'secondServePointsWon0', 'breakPointsFaced0', 'breakPointsSaved0',
                                    'serviceGamePlayed0', 'serviceGamesWon0', 'totalServicePointsWon0',
                                    'firstServeReturnPointsWon0', 'secondServeReturnPointsWon0',
                                    'breakPointsOpportunities0', 'breakPointsConverted0', 'returnGamesPlayed0',
                                    'returnGamesWon0', 'returnPointsWon0', 'totalPointsWon0', 'url0', 'year0',
                                    'weight0', 'height0', 'age0', 'startCareer0', 'plays0', 'ranking0', 'points0',
                                    'tournCell0', 'pointsDropping0', 'nextBest0', 'surface0']

            joueur1Stats = getCorrespondingStats(joueur1, surface)

            forme1, winMean1 = getFormeAndRankMean(get_corresponding_player(joueur1, allPlayers))

            joueur1Stats.columns = ['player1', 'aces1', 'doubleFautes1', 'firstServe1', 'firstServePointsWon1',
                                    'secondServePointsWon1', 'breakPointsFaced1', 'breakPointsSaved1',
                                    'serviceGamePlayed1', 'serviceGamesWon1', 'totalServicePointsWon1',
                                    'firstServeReturnPointsWon1', 'secondServeReturnPointsWon1',
                                    'breakPointsOpportunities1', 'breakPointsConverted1', 'returnGamesPlayed1',
                                    'returnGamesWon1', 'returnPointsWon1', 'totalPointsWon1', 'url1', 'year1',
                                    'weight1', 'height1', 'age1', 'startCareer1', 'plays1', 'ranking1', 'points1',
                                    'tournCell1', 'pointsDropping1', 'nextBest1', 'surface1']

            versus = getPlayersConfrontation(joueur0, joueur1, surface, joueur0Stats.ranking0, joueur1Stats.ranking1)
            versus = pd.DataFrame([versus['player0']['player0Rate'], versus['player0']['player0RateSurface'],
                                   versus['player1']['player1Rate'], versus['player1']['player1RateSurface']]).T

            versus.columns = ['RateFace2Face0', 'RateFace2FaceSurface0', 'RateFace2Face1', 'RateFace2FaceSurface1']

            current_match = pd.concat([joueur0Stats.reset_index(), joueur1Stats.reset_index(), versus], axis=1)

            current_match['plays0'] = [str(play.split(',')[0].lstrip().rstrip()) for play in current_match.plays0]

            current_match['plays1'] = [str(play.split(',')[0].lstrip().rstrip()) for play in current_match.plays1]

            current_match['aces0'] = (int(current_match['aces0'].iloc[0])) / (
                        3.5 * int(current_match.serviceGamePlayed0.iloc[0]))
            current_match['aces1'] = (int(current_match['aces1'].iloc[0])) / (
                        3.5 * int(current_match.serviceGamePlayed1.iloc[0]))

            current_match['doubleFautes0'] = (int(current_match['doubleFautes0'].iloc[0])) / (
                        3.5 * int(current_match.serviceGamePlayed0.iloc[0]))
            current_match['doubleFautes1'] = (int(current_match['doubleFautes1'].iloc[0])) / (
                        3.5 * int(current_match.serviceGamePlayed0.iloc[0]))

            all_plays = all_plays.append(pd.concat([returnPlays(current_match.plays0.iloc[0], 0),
                                                    returnPlays(current_match.plays1.iloc[0], 1)], axis=0),
                                         ignore_index=True)

            allInOut = allInOut.append(returnInOut(inout), ignore_index=True)
            allSurface = allSurface.append(returnSurface(surface), ignore_index=True)
            AllForme0.append(forme0)
            AllForme1.append(forme1)
            AllWinMean0.append(winMean0)
            AllWinMean1.append(winMean1)

            allTypeTournement = allTypeTournement.append(pd.DataFrame([typeTournement], columns=['typeTournament']),
                                                         ignore_index=True)

            all_match_to_predict = all_match_to_predict.append(current_match)

            matchSelected.append([joueur0, joueur1, currentMatch['start'],currentMatch['oddJoueur0'],currentMatch['oddJoueur1']])
            
        except Exception as e:
            print(e)

    all_match_to_predict = pd.concat([all_match_to_predict.reset_index(), all_plays.reset_index(),
                                      allInOut.reset_index(), allSurface.reset_index(),
                                      allTypeTournement.reset_index(),
                                      pd.DataFrame(AllForme0, columns=['Forme0']),
                                      pd.DataFrame(AllForme1, columns=['Forme1']),
                                      pd.DataFrame(AllWinMean0, columns=['WinMean0']),
                                      pd.DataFrame(AllWinMean1, columns=['WinMean1'])], axis=1)

    all_match_to_predict.RateFace2Face0 = pd.to_numeric(all_match_to_predict.RateFace2Face0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.RateFace2Face1 = pd.to_numeric(all_match_to_predict.RateFace2Face1.str.replace(r'[^0-9]+', ''))

    all_match_to_predict.age0 = [2020 - datetime.strptime(age[age.find("(") + 1:age.find(")")], '%Y.%m.%d').date().year
                                 for age in all_match_to_predict.age0]
    all_match_to_predict.age1 = [2020 - datetime.strptime(age[age.find("(") + 1:age.find(")")], '%Y.%m.%d').date().year
                                 for age in all_match_to_predict.age1]

    all_match_to_predict.breakPointsConverted0 = pd.to_numeric(
        all_match_to_predict.breakPointsConverted0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.breakPointsSaved0 = pd.to_numeric(
        all_match_to_predict.breakPointsSaved0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.firstServe0 = pd.to_numeric(all_match_to_predict.firstServe0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.firstServePointsWon0 = pd.to_numeric(
        all_match_to_predict.firstServePointsWon0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.firstServeReturnPointsWon0 = pd.to_numeric(
        all_match_to_predict.firstServeReturnPointsWon0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.serviceGamesWon0 = pd.to_numeric(
        all_match_to_predict.serviceGamesWon0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.totalPointsWon0 = pd.to_numeric(
        all_match_to_predict.totalPointsWon0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.totalServicePointsWon0 = pd.to_numeric(
        all_match_to_predict.totalServicePointsWon0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.height0 = pd.to_numeric(all_match_to_predict.height0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.returnGamesWon0 = pd.to_numeric(
        all_match_to_predict.returnGamesWon0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.returnPointsWon0 = pd.to_numeric(
        all_match_to_predict.returnPointsWon0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.secondServePointsWon0 = pd.to_numeric(
        all_match_to_predict.secondServePointsWon0.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.secondServeReturnPointsWon0 = pd.to_numeric(
        all_match_to_predict.secondServeReturnPointsWon0.str.replace(r'[^0-9]+', ''))

    all_match_to_predict.breakPointsConverted1 = pd.to_numeric(
        all_match_to_predict.breakPointsConverted1.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.breakPointsSaved1 = pd.to_numeric(
        all_match_to_predict.breakPointsSaved1.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.firstServe1 = pd.to_numeric(all_match_to_predict.firstServe1.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.firstServePointsWon1 = pd.to_numeric(
        all_match_to_predict.firstServePointsWon1.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.firstServeReturnPointsWon1 = pd.to_numeric(
        all_match_to_predict.firstServeReturnPointsWon1.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.serviceGamesWon1 = pd.to_numeric(
        all_match_to_predict.serviceGamesWon1.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.totalPointsWon1 = pd.to_numeric(
        all_match_to_predict.totalPointsWon1.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.totalServicePointsWon1 = pd.to_numeric(
        all_match_to_predict.totalServicePointsWon1.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.height1 = pd.to_numeric(all_match_to_predict.height1.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.returnGamesWon1 = pd.to_numeric(
        all_match_to_predict.returnGamesWon1.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.returnPointsWon1 = pd.to_numeric(
        all_match_to_predict.returnPointsWon1.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.secondServePointsWon1 = pd.to_numeric(
        all_match_to_predict.secondServePointsWon1.str.replace(r'[^0-9]+', ''))
    all_match_to_predict.secondServeReturnPointsWon1 = pd.to_numeric(
        all_match_to_predict.secondServeReturnPointsWon1.str.replace(r'[^0-9]+', ''))

    all_match_to_predict.breakPointsConverted0 = all_match_to_predict.breakPointsConverted0.replace('[]', np.NaN)
    all_match_to_predict.breakPointsFaced0 = all_match_to_predict.breakPointsFaced0.replace('[]', np.NaN)
    all_match_to_predict.breakPointsOpportunities0 = all_match_to_predict.breakPointsOpportunities0.replace('[]',
                                                                                                            np.NaN)
    all_match_to_predict.breakPointsSaved0 = all_match_to_predict.breakPointsSaved0.replace('[]', np.NaN)
    all_match_to_predict.doubleFautes0 = all_match_to_predict.doubleFautes0.replace('[]', np.NaN)
    all_match_to_predict.firstServe0 = all_match_to_predict.firstServe0.replace('[]', np.NaN)
    all_match_to_predict.firstServePointsWon0 = all_match_to_predict.firstServePointsWon0.replace('[]', np.NaN)
    all_match_to_predict.firstServeReturnPointsWon0 = all_match_to_predict.firstServeReturnPointsWon0.replace('[]',
                                                                                                              np.NaN)

    all_match_to_predict.breakPointsConverted1 = all_match_to_predict.breakPointsConverted1.replace('[]', np.NaN)
    all_match_to_predict.breakPointsFaced1 = all_match_to_predict.breakPointsFaced1.replace('[]', np.NaN)
    all_match_to_predict.breakPointsOpportunities1 = all_match_to_predict.breakPointsOpportunities1.replace('[]',
                                                                                                            np.NaN)
    all_match_to_predict.breakPointsSaved1 = all_match_to_predict.breakPointsSaved1.replace('[]', np.NaN)
    all_match_to_predict.doubleFautes1 = all_match_to_predict.doubleFautes1.replace('[]', np.NaN)
    all_match_to_predict.firstServe1 = all_match_to_predict.firstServe1.replace('[]', np.NaN)
    all_match_to_predict.firstServePointsWon1 = all_match_to_predict.firstServePointsWon1.replace('[]', np.NaN)
    all_match_to_predict.firstServeReturnPointsWon1 = all_match_to_predict.firstServeReturnPointsWon1.replace('[]',
                                                                                                              np.NaN)

    all_match_to_predict.breakPointsFaced0 = pd.to_numeric(all_match_to_predict.breakPointsFaced0)
    all_match_to_predict.breakPointsConverted0 = pd.to_numeric(all_match_to_predict.breakPointsConverted0)
    all_match_to_predict.returnGamesWon0 = pd.to_numeric(all_match_to_predict.returnGamesWon0)
    all_match_to_predict.serviceGamesWon0 = pd.to_numeric(all_match_to_predict.serviceGamesWon0)

    all_match_to_predict.breakPointsFaced1 = pd.to_numeric(all_match_to_predict.breakPointsFaced1)
    all_match_to_predict.breakPointsConverted1 = pd.to_numeric(all_match_to_predict.breakPointsConverted1)
    all_match_to_predict.returnGamesWon1 = pd.to_numeric(all_match_to_predict.returnGamesWon1)
    all_match_to_predict.serviceGamesWon1 = pd.to_numeric(all_match_to_predict.serviceGamesWon1)

    all_match_to_predict.points0 = [point.replace(",", ".") for point in all_match_to_predict.points0]
    all_match_to_predict.points1 = [point.replace(",", ".") for point in all_match_to_predict.points1]

    all_match_to_predict.pointsDropping0 = [point.replace(",", ".") for point in all_match_to_predict.pointsDropping0]
    all_match_to_predict.pointsDropping1 = [point.replace(",", ".") for point in all_match_to_predict.pointsDropping1]

    all_match_to_predict[
        'mental0'] = all_match_to_predict.breakPointsFaced0 + all_match_to_predict.breakPointsConverted0 + all_match_to_predict.returnGamesWon0 + all_match_to_predict.serviceGamesWon0
    all_match_to_predict[
        'mental1'] = all_match_to_predict.breakPointsFaced1 + all_match_to_predict.breakPointsConverted1 + all_match_to_predict.returnGamesWon1 + all_match_to_predict.serviceGamesWon1

    all_match_to_predict = all_match_to_predict.drop(['index', 'player0', 'url0', 'year0', 'weight0', 'startCareer0',
                                                      'index', 'player1', 'url1', 'year1', 'weight1', 'startCareer1'],
                                                     axis=1)

    return all_match_to_predict, matchSelected


all_match_to_predict_clean, matchSelected = buildDatasetForPrediction()

all_match_to_predict_clean

joueur 0 baena carballes
joueur 1 marco cecchinato
joueur 0 andrej martin
joueur 1 facundo bagnis
joueur 0 corentin moutet
joueur 1 jozef kovalik
joueur 0 fernando verdasco
joueur 1 portero martinez
joueur 0 francisco cerundolo
joueur 1 laslo djere
joueur 0 borna coric
joueur 1 thiago monteiro
joueur 0 diego schwartzman
joueur 1 federico delbonis
joueur 0 go soeda
joueur 1 woo soon
joueur 0 kyle edmund
joueur 1 yasutaka uchiyama
joueur 0 danilo petrovic
joueur 1 paolo lorenzi
'NoneType' object has no attribute 'columns'
joueur 0 brayden schnur
joueur 1 dominik koepfer
joueur 0 henri laaksonen
joueur 1 yoshihito nishioka
joueur 0 brian shi
joueur 1 cameron norrie
invalid literal for int() with base 10: '[]'
joueur 0 jason jung
joueur 1 kevin anderson
joueur 0 marcos giron
joueur 1 ugo humbert
joueur 0 andreas seppi
joueur 1 steve johnson
joueur 0 hubert hurkacz
joueur 1 stefanos tsitsipas
joueur 0 filip krajinovic
joueur 1 tallon griekspoor
joueur 0 gilles simon
joueur 1 mikhail kukushk

,level_0,aces0,doubleFautes0,firstServe0,firstServePointsWon0,secondServePointsWon0,breakPointsFaced0,breakPointsSaved0,serviceGamePlayed0,serviceGamesWon0,...,Clay,Grass,Hard,typeTournament,Forme0,Forme1,WinMean0,WinMean1,mental0,mental1
0,0,0.066981,0.073260,65,67,51,174,63,273,76,...,1.0,0.0,0.0,250,0.500000,0.333333,150.0,200.0,325,269
1,0,0.000000,0.035714,71,54,31,10,50,8,38,...,1.0,0.0,0.0,250,0.500000,0.666667,150.0,71.5,137,177
2,0,0.046259,0.103401,68,61,49,75,49,105,64,...,1.0,0.0,0.0,250,0.666667,0.500000,200.0,150.0,221,167
3,0,0.086619,0.082741,65,69,47,150,65,221,76,...,1.0,0.0,0.0,250,0.333333,0.333333,200.0,200.0,282,94
4,0,0.000000,0.131868,66,60,42,11,55,13,62,...,1.0,0.0,0.0,250,0.333333,0.333333,200.0,200.0,146,314
5,0,0.094300,0.050669,60,74,55,79,57,203,83,...,1.0,0.0,0.0,250,0.500000,0.333333,150.0,200.0,220,184
6,0,0.026997,0.077615,66,63,48,186,52,254,65,...,1.0,0.0,0.0,250,0.333333,0.333333,200.0,200.0,332,318
7,0,0.109890,0.054945,54,73,49,16,69,26,81,...,0.0,0.0,1.0,250,0.666667,0.333333,200.0,200.0,147,182
8,0,0.155948,0.050085,61,73,52,83,47,251,82,...,0.0,0.0,1.0,250,0.333333,0.333333,200.0,200.0,217,196
9,0,0.204082,0.063492,69,72,45,79,68,126,80,...,0.0,0.0,1.0,250,0.500000,0.333333,150.0,200.0,200,190


In [141]:
all_match_to_predict_clean.

0      80
1     107
2      83
3      49
4     248
5      28
6      14
7     121
8      69
9     106
10    104
11    127
12    102
13     72
14     37
15     40
16     55
17     78
18     11
19     10
20     42
21      5
Name: ranking0, dtype: int64

In [137]:
all_match_to_predict_clean = pd.concat([pd.DataFrame(np.zeros(len(all_match_to_predict_clean)),columns=['label']),
                                        all_match_to_predict_clean.reset_index()], axis=1).drop('index',axis=1)


all_match_to_predict_clean = all_match_to_predict_clean[['typeTournament', 'label', 'aces0', 'age0', 'breakPointsConverted0',
       'breakPointsFaced0', 'breakPointsOpportunities0', 'breakPointsSaved0',
       'doubleFautes0', 'firstServe0', 'firstServePointsWon0',
       'firstServeReturnPointsWon0', 'height0', 'nextBest0', 'points0',
       'pointsDropping0', 'ranking0', 'returnGamesPlayed0', 'returnGamesWon0',
       'returnPointsWon0', 'secondServePointsWon0',
       'secondServeReturnPointsWon0', 'serviceGamePlayed0', 'serviceGamesWon0',
       'totalPointsWon0', 'totalServicePointsWon0', 'tournCell0', 'aces1',
       'age1', 'breakPointsConverted1', 'breakPointsFaced1',
       'breakPointsOpportunities1', 'breakPointsSaved1', 'doubleFautes1',
       'firstServe1', 'firstServePointsWon1', 'firstServeReturnPointsWon1',
       'height1', 'nextBest1', 'points1', 'pointsDropping1', 'ranking1',
       'returnGamesPlayed1', 'returnGamesWon1', 'returnPointsWon1',
       'secondServePointsWon1', 'secondServeReturnPointsWon1',
       'serviceGamePlayed1', 'serviceGamesWon1', 'totalPointsWon1',
       'totalServicePointsWon1', 'tournCell1', 'mental0', 'mental1', 'Indoor',
       'Outdoor', 'Carpet', 'Clay', 'Grass', 'Hard', 'Left-Handed',
       'Right-Handed', 'Left-Handed.1', 'Right-Handed.1', 'RateFace2Face0',
       'RateFace2FaceSurface0', 'RateFace2Face1', 'RateFace2FaceSurface1',
       'Forme0', 'Forme1', 'WinMean0', 'WinMean1']]

all_match_to_predict_clean.RateFace2Face0 = all_match_to_predict_clean.RateFace2Face0.fillna(50)
all_match_to_predict_clean.RateFace2Face1 = all_match_to_predict_clean.RateFace2Face1.fillna(50)

f2f0_list = []
f2f1_list = []

for f2f0, f2f1 in zip(all_match_to_predict_clean.RateFace2Face0,all_match_to_predict_clean.RateFace2Face1):
    if f2f0+f2f1 == 1:
        f2f0_list.append(f2f0)
        f2f1_list.append(f2f1)
    elif f2f0+f2f1 == 100:
        f2f0_list.append(f2f0/100)
        f2f1_list.append(f2f1/100)
    else:
        f2f0_list.append(0.50)
        f2f1_list.append(0.50)
        

all_match_to_predict_clean.RateFace2Face0 = f2f0_list
all_match_to_predict_clean.RateFace2Face1 = f2f1_list


f2f0S_list = []
f2f1S_list = []

for f2f0S, f2f1S in zip(all_match_to_predict_clean.RateFace2FaceSurface0,all_match_to_predict_clean.RateFace2FaceSurface1):
    if f2f0S+f2f1S == 1:
        f2f0S_list.append(f2f0S)
        f2f1S_list.append(f2f1S)
    elif f2f0S+f2f1S == 100:
        f2f0S_list.append(f2f0S/100)
        f2f1S_list.append(f2f1S/100)
    else:
        f2f0S_list.append(0.50)
        f2f1S_list.append(0.50)
        

all_match_to_predict_clean.RateFace2FaceSurface0 = f2f0S_list
all_match_to_predict_clean.RateFace2FaceSurface1 = f2f1S_list

all_match_to_predict_clean.columns

Index(['typeTournament', 'label', 'label', 'aces0', 'age0',
       'breakPointsConverted0', 'breakPointsFaced0',
       'breakPointsOpportunities0', 'breakPointsSaved0', 'doubleFautes0',
       'firstServe0', 'firstServePointsWon0', 'firstServeReturnPointsWon0',
       'height0', 'nextBest0', 'points0', 'pointsDropping0', 'ranking0',
       'returnGamesPlayed0', 'returnGamesWon0', 'returnPointsWon0',
       'secondServePointsWon0', 'secondServeReturnPointsWon0',
       'serviceGamePlayed0', 'serviceGamesWon0', 'totalPointsWon0',
       'totalServicePointsWon0', 'tournCell0', 'aces1', 'age1',
       'breakPointsConverted1', 'breakPointsFaced1',
       'breakPointsOpportunities1', 'breakPointsSaved1', 'doubleFautes1',
       'firstServe1', 'firstServePointsWon1', 'firstServeReturnPointsWon1',
       'height1', 'nextBest1', 'points1', 'pointsDropping1', 'ranking1',
       'returnGamesPlayed1', 'returnGamesWon1', 'returnPointsWon1',
       'secondServePointsWon1', 'secondServeReturnPointsW

In [134]:
from pickle import load
scaler = load(open('data/scaler.pkl', 'rb'))
# transform the test dataset
X_predict_scaled = pd.DataFrame(scaler.transform(all_match_to_predict_clean), columns=all_match_to_predict_clean.columns)

In [138]:
all_match_to_predict_clean.shape

(22, 73)

In [132]:
#X_predict_scaled = pd.DataFrame(X_predict_scaled, columns=all_match_to_predict_clean.columns)
X_predict_scaled.RateFace2Face0 = X_predict_scaled.RateFace2Face0/100
X_predict_scaled.RateFace2Face1 = X_predict_scaled.RateFace2Face1/100

In [129]:
from pickle import load
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
import os


# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=2):
  dataframe = dataframe.drop(['label'],axis=1)
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe)))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 2 # A small batch sized is used for demonstration purposes
predict_ds = df_to_dataset(X_predict_scaled, batch_size=batch_size)

print(len(X_predict_scaled.columns))




AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [86]:
def create_model():
        
    model = tf.keras.Sequential([
        feature_layer,
        layers.Dense(1000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2000, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    opt = tf.keras.optimizers.SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
    model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    return model


model = create_model()
  
model.load_weights('./checkpoint/my_checkpoint')

prediction = model.predict(predict_ds)

In [140]:
prediction_with_names = pd.concat([pd.DataFrame(matchSelected, columns=['joueur0','joueur1','Date','Odd0','Odd1']),
           pd.DataFrame(prediction, columns=['prediction'])],axis=1)
prediction_with_names

,joueur0,joueur1,Date,Odd0,Odd1,prediction
0,baena carballes,marco cecchinato,2020-02-11T17:33:55Z,2.400000,1.520000,0.574112
1,andrej martin,facundo bagnis,2020-02-11T17:36:13Z,6.250000,1.111111,0.372223
2,corentin moutet,jozef kovalik,2020-02-11T17:37:40Z,3.900000,1.240000,0.205501
3,fernando verdasco,portero martinez,2020-02-11T19:10:00Z,1.550000,2.400000,0.615435
4,francisco cerundolo,laslo djere,2020-02-11T20:25:00Z,1.350000,3.200000,0.384957
5,borna coric,thiago monteiro,2020-02-12T17:30:00Z,2.080000,1.750000,0.544838
6,diego schwartzman,federico delbonis,2020-02-12T17:30:00Z,1.200000,4.500000,0.459384
7,go soeda,woo soon,2020-02-11T17:03:42Z,2.200000,1.615385,0.487824
8,kyle edmund,yasutaka uchiyama,2020-02-11T18:45:40Z,1.250000,3.800000,0.417298
9,brayden schnur,dominik koepfer,2020-02-11T19:55:00Z,2.400000,1.550000,0.582968


In [89]:
prediction_with_names[prediction_with_names.prediction>0.6]

,joueur0,joueur1,Date,Odd0,Odd1,prediction
3,acosta diaz,cristian garin,2020-02-11T18:50:00Z,7.500000,1.1,0.615435
10,kyle edmund,yasutaka uchiyama,2020-02-11T18:20:00Z,1.250000,3.9,0.692562
23,daniil medvedev,vasek pospisil,2020-02-12T18:30:00Z,1.166667,5.0,0.836938


In [90]:
prediction_with_names[prediction_with_names.prediction<0.4]

,joueur0,joueur1,Date,Odd0,Odd1,prediction
1,andrej martin,facundo bagnis,2020-02-11T17:30:00Z,1.650000,2.200000,0.372223
2,corentin moutet,jozef kovalik,2020-02-11T17:30:00Z,1.571429,2.375000,0.205501
4,fernando verdasco,portero martinez,2020-02-11T18:50:00Z,1.520000,2.500000,0.384957
11,henri laaksonen,yoshihito nishioka,2020-02-11T19:40:00Z,1.250000,3.900000,0.083020
12,brayden schnur,dominik koepfer,2020-02-11T19:40:00Z,2.400000,1.550000,0.397860
13,jason jung,kevin anderson,2020-02-12T01:50:00Z,4.000000,1.240000,0.255388
14,marcos giron,ugo humbert,2020-02-12T17:00:00Z,4.250000,1.222222,0.286993
17,hubert hurkacz,stefanos tsitsipas,2020-02-11T18:30:00Z,3.100000,1.363636,0.383483
19,gilles simon,mikhail kukushkin,2020-02-12T10:00:00Z,1.909091,1.850000,0.149949
20,jannik sinner,radu albot,2020-02-12T10:00:00Z,1.330000,3.350000,0.238990


In [101]:
    import re

    def upload_prediction_to_AWS(row):

        date = row['Date']

        id_1 = row['joueur1']
        id_0 = row['joueur0']

        id_match = str(re.sub("[^0-9]", "", date))+str(joueur0)+str(joueur1)

        prob_0 = str(1-row.prediction)
        prob_1 = str(row.prediction)
        prob_draw = "0"

        if prob_0 > prob_1:
            winner = id_0
        else:
            winner = id_1

        odd_0 = str(row.Odd0)
        odd_1 = str(row.Odd1)
        odd_draw = "0"

        try:
            connection = psycopg2.connect(
                database="pronoodb",
                user="pronoodbPostgres",
                password="pronoodb2019",
                host="pronoodb.cutflievswqr.eu-central-1.rds.amazonaws.com",
                port='5432'
            )
            cursor = connection.cursor()

            postgres_insert_query = """ INSERT INTO pronoodb.tennismatches (player_0_name,player_1_name,winner_name,probability_score_player_0, 
                                                                      probability_score_player_1, odd_player_0, odd_player_1, is_visible_to_users,
                                                                      match_date) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
            record_to_insert = (id_0, id_1, winner, prob_0, prob_1,odd_0, odd_1, True, date)
            cursor.execute(postgres_insert_query, record_to_insert)

            connection.commit()
            count = cursor.rowcount
            print (count, "Record inserted successfully into match table")

        except (Exception, psycopg2.Error) as error :
            if(connection):
                print("Failed to insert record into matches table", error)

        finally:
            #closing database connection.
            if(connection):
                cursor.close()
                connection.close()
                print("PostgreSQL connection is closed")
                
                
upload_prediction_to_AWS(prediction_with_names[prediction_with_names.prediction>0.6].iloc[0])

1 Record inserted successfully into match table
PostgreSQL connection is closed


In [114]:
for index, row in prediction_with_names.iterrows():
    if 1.2<row.Odd0 and row.Odd0<3.5 and 1.2<row.Odd1 and row.Odd1<3.5:
        if row.prediction<0.4 or row.prediction>0.6:
            upload_prediction_to_AWS(row)
            


1 Record inserted successfully into match table
PostgreSQL connection is closed
1 Record inserted successfully into match table
PostgreSQL connection is closed
1 Record inserted successfully into match table
PostgreSQL connection is closed
1 Record inserted successfully into match table
PostgreSQL connection is closed
1 Record inserted successfully into match table
PostgreSQL connection is closed
1 Record inserted successfully into match table
PostgreSQL connection is closed
1 Record inserted successfully into match table
PostgreSQL connection is closed


In [1550]:
getInfoTournois("Montpellier")

hard


('hard', 'Indoor', 250)

In [1571]:
all_tournements_forDNN = all_tournements_forDNN.dropna()

In [1598]:
dnnPlayerName = set(list(set(all_tournements_forDNN.player0))+list(set(all_tournements_forDNN.player1)))

getFormPlayer(get_corresponding_player('filip krajinovic', dnnPlayerName)).year

26643    2010.0
26687    2010.0
27263    2010.0
27264    2010.0
27265    2010.0
          ...  
55566    2019.0
55683    2019.0
55704    2019.0
55705    2019.0
55834    2019.0
Name: year, Length: 137, dtype: float64

NameError: name 'prediction_with_names' is not defined

In [150]:
all_tournaments_with_stat.height0.str.extract('(\d+)')

,0
0,120
1,150


AttributeError: 'DataFrame' object has no attribute 'extract'

In [145]:
all_tournaments_with_stat = pd.DataFrame(['(120cm)','(150cm)'], columns=['height0'])

In [155]:
height0 = all_tournaments_with_stat.height0.str.extract('(\d+)')
height0.columns = ['height0']
height0

,height0
0,120
1,150
